# The GPT Language Model

## Imports

Here are the packages we need to import.

In [1]:
from nlpmodels.models import gpt
from nlpmodels.utils import train,utils,gpt_sampler
from nlpmodels.utils.elt import gpt_dataset
from argparse import Namespace
utils.set_seed_everywhere()

## Language Model: WikiText2

We will try to train our transformer model to learn how to predict the next word in torchtext WikiText2 database.
I took the first 300k from the training set to reduce computation time.

### Hyper-parameters

These are the data processing and model training hyper-parameters for this run. Note that we are running a smaller model
than cited in the paper for fewer iterations...on a CPU. This is meant merely to demonstrate it works.

In [3]:
args = Namespace(
        # Model hyper-parameters
        num_layers_per_stack=2,  # original value = 12
        dim_model=12, #original value = 768
        dim_ffn=48, # original value = 3072
        num_heads=2, # original value = 12
        block_size=64, # original value = 512, context window
        dropout=0.1,
        # Training hyper-parameters
        num_epochs=1, #obviously super short
        learning_rate=0.0,
        batch_size=32, #original value = 64
    )

In [4]:
train_loader, vocab = gpt_dataset.GPTDataset.get_training_dataloader(args)
model = gpt.GPT(vocab_size = len(vocab),
            num_layers_per_stack= args.num_layers_per_stack,
            dim_model = args.dim_model,
            dim_ffn = args.dim_ffn,
            num_heads = args.num_heads,
            block_size = args.block_size,
            dropout = args.dropout)
trainer = train.GPTTrainer(args,vocab.mask_index,model,train_loader,vocab)

1lines [00:00,  3.74lines/s]


# Self-supervised training
Now we will run the first step in GPT training process, where we train the model to
maximize the objective

```
max p(x[k]|x[k-1],[k-2],...x[k-block_size])
```.

This is an unsupervised (more aptly described as "self-supervised") loss. After this model is trained,
we can run then continue it onto another problem (can freeze layers to only continue training the top layers).

In [5]:
trainer.run()

[Epoch 0]: 100%|██████████| 9375/9375 [2:30:30<00:00,  1.04it/s, loss=5.17]  


Finished Training...


Note that this model was run for a **very** short period of time. The goal is just to show how this works - you can
play with the hyper-parameters as you see fit.
We only ran for 1 epoch, on a much smaller model,
with a smaller dataset than was suggested in the paper.

Let's see if the output the model completes makes any sense.

# GPT Completes A Sequence

In the spirit of Kaparthy's minGPT::play_char notebook, we can use a greedy_sampler to see how the model
continues a sequence.

In [60]:
import torch



prompt = "ernest hemingway first novel , the sun also rises , " \
         "treats of certain of those younger americas concerning whom gertrude stein has remarked :" \
         " you are all a lost generation . this is the novel for which a keen appetite was stimulated by" \
         " mr . hemingway 's exciting volume of short stories. " \
         " the clear objectivity and the sustained intensity of the stories , " \
         "and their concentration upon action in the present moment, seemed to point to a failure to project " \
         "a novel in terms of the same method, yet a resort to any other method would have let down the " \
         "reader's expectations. it is a relief to find that the sun also rises maintains the same heightened , " \
         "intimate tangibility as the shorter narratives and does it in the same kind of weighted, quickening prose. "
prompt_tensor = torch.LongTensor([[vocab.lookup_token(s) for s in prompt.split(" ")]])
prompt_tensor_batch = trainer._reformat_data((prompt_tensor,None))
steps = 64
y_hat_indices = gpt_sampler.sampler(model=model, data=prompt_tensor_batch,
                                          steps=steps,block_size=64,do_sample=True).src
y_hat_tokens = ' '.join([vocab.lookup_index(int(idx)) for idx in y_hat_indices[0]])

Here is the prompt it was provided, a review of Ernest Hemingway's novel, The Sun Also Rises:

In [32]:
for idx in range(0,len(prompt.split(" ")),8):
    print(" ".join(prompt.split(" ")[idx:idx+8]))

ernest hemingway first novel , the sun also
rises , treats of certain of those younger
americas concerning whom gertrude stein has remarked :
you are all a lost generation . this
is the novel for which a keen appetite
was stimulated by mr . hemingway 's exciting
volume of short stories.  the clear objectivity
and the sustained intensity of the stories ,
and their concentration upon action in the present
moment, seemed to point to a failure to
project a novel in terms of the same
method, yet a resort to any other method
would have let down the reader's expectations. it
is a relief to find that the sun
also rises maintains the same heightened , intimate
tangibility as the shorter narratives and does it
in the same kind of weighted, quickening prose.



Working off that sequence, here is how the model completed the next 64 words in this review:

In [61]:
for idx in range(0,len(y_hat_tokens.split(" ")),8):
    print(" ".join(y_hat_tokens.split(" ")[idx:idx+8]))

<unk> were started to <unk> olajuwon has been
bed at in polish jazz @-@ overall films
. around his violent several name or from
the community in some vessels , passengers is
being to direct works gas nuclear works of
relationships and he believes that who chose taken
the nba party , germany with distinguished war
and on a representation . kevin bull ,
but


Well, as expected... this doesn't make any sense really. Pockets of words make sense, but overall it does not.
As the language model is trained further and using more parameters, we would expect the believability of this text to increase.

# Supervised training

Once the model is trained in the self-supervised phase, go forth and apply it to a different problem!